# Algorithmic Trading - Moving Average Convergence Divergence

This work will explain the Moving Average Convergence Divergence (MACD) strategy, providing the implementation code and its interpretation.

## 1) Theoretical Concept

The MACD is a technical analysis indicator used in finance as a trading strategy. Developped by Gerard Appel in the 1970s, this method provide information about dynamic of buying and selling stocks in order to take profit of it. There are several uses for this method. 

In order to apply the studied version, it requires to compute the MACD and a signal trend. The MACD is calculated using two exponantial moving average. It represents the difference between the value of a slower Exponential Moving Average (EMA) from a faster EMA. In other words, an EMA with a longer time period is substracted from an EMA with a shorter time period. The signal trend in found by computing the EMA with an even shorter time span.

The EMA and MACD are computed as follows: 

$$ EMA(n) = \frac{2}{1 + n} \times(P_t - EMA_{t-1}) + EMA_{t-1} $$

$$ MACD_t = EMA(s)_t - EMA(l)_t $$

where $P_t$ is the closing price of a stock on day $t$, and $n$ is the number of periods for calculating $EMA(n)$. $EMA(s)$ and $EMA(l)$ are the $EMA(n)$ for a short and longer time period, respectively.

In general, the period's lenght of the short, long EMA and the EMA are 12, 26 and 9, respectively.

The graphic representation of the values of MACD and the EMA signal is used to determine the position of the trader. In fact, it is suggest to buy the stock when the MACD line crosses above the signal line. The stock should be sold when the MACD line crosses below the signal line. 

## 2) Implementation

To begin, all the libraries that will be used must be imported.

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import timedelta, date

Then, the function which compute the EMA's and MACD and collect is definied. The hole function is divided into smaller piece in order to provide an explaination.

**Step 1:** The function requires 6 inputs.

**Step 2:** The code download the data related to a specific stock over a given period. The following lines compute the Exponential Moving Averages (EMA) and the Moving Average Convergence Divergence (MACD) based on the close value of the stock. The function *.ewm* allows the calculation of the exponential moving average. The component *span* replaces $\frac{2}{1 + n}$ in the equation above. The component *adjusted=False* signifies that the computation of the exponentially weighted function is done recursively. All results are stored in a new data frame with labeled columns.

**Step 3:** A loop iterates over all results to check which lines of the data frame meet the specific conditions. If a line satisies the condition, the required information is stored in new lists. Since the EMA's and the MACD are calculated based on the close value, and the signal is also defined based on the close value of the stock, the buy or sell of the stock occurs the following day. Therefore, the price at which the share is bought, i.e. the next day is retrieved using the function *.iloc*.

**Step 4:** It may occur that the first signal is sell or the last signal is buy. However, it is not possible to compute the return for the period before the first sell or after the last buy, as it goes beyond the scope of the analysis. This problem arises because the time period may be too short. The fourth step is designed to rectify these mistakes.

**Step 5:** In this step, the return for each holding period is computed. These values are also stored in a new list.

**Step 6:** The outputs are set.

In [ ]:
# Step 1
def Strategy(ticker, start, end, short_window, long_window, signal_window):

# Step_2
    df = yf.download(ticker, start, end)
    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()

# Step 3
    signal_buy, signal_sell = [], []
    signal_buy_dates, signal_sell_dates = [], []
    buy_prices, sell_prices = [], []
    buy_dates, sell_dates = [], []
    
    for i in range(1, len(df)-1):
        if (df['MACD'].iloc[i] > df['signal'].iloc[i]) and (df['MACD'].iloc[i-1] < df['signal'].iloc[i-1]):
            signal_buy.append(i)
            signal_buy_dates.append(df.index[i])
            buy_prices.append(df['Open'].iloc[i+1])
            buy_dates.append(df.index[i+1])
        elif (df['MACD'].iloc[i] < df['signal'].iloc[i]) and (df['MACD'].iloc[i-1] > df['signal'].iloc[i-1]):
            signal_sell.append(i)
            signal_sell_dates.append(df.index[i])
            sell_prices.append(df['Open'].iloc[i+1])
            sell_dates.append(df.index[i+1])
# Step 4            
    if signal_sell and signal_buy:
        if signal_sell[0] < signal_buy[0]:
            signal_sell = signal_sell[1:]
            signal_sell_dates = signal_sell_dates[1:]
            sell_prices = sell_prices[1:]
            sell_dates = sell_dates[1:]
        
        if signal_sell[-1] < signal_buy[-1]:
            signal_buy = signal_buy[:-1]
            signal_buy_dates = signal_buy_dates[:-1]
            buy_prices = buy_prices[:-1]
            buy_dates = buy_dates[:-1]
# Step 5        
    return_performance = []

    for i in range(len(sell_prices)):
        return_performance.append(((sell_prices[i] - buy_prices[i]) / buy_prices[i]) * 100)

# Step 6
    output = (df, signal_buy, signal_buy_dates, buy_prices, buy_dates, signal_sell, signal_sell_dates, 
              sell_prices, sell_dates, return_performance)
    
    return output


## 3) Application

The strategy will be applied using data from the preceding 365 days. Therefore, the following code examines today's data and computes the relevant period.

In [ ]:
today = date.today()
end_period = today - timedelta(days = 1)
start_period = today - timedelta(days = 366)
print("The first day considered in our computation is", start_period, "and the last day is", end_period)

The function *Strategy* is used. All required inputs are set. It this case, the strategy is applied for *McDonald's (MCD)*.

In [ ]:
result = Strategy(ticker='MCD', start=start_period ,end=end_period,short_window=12,long_window=26,
    signal_window=9)

A command is executed to display the database containing the computed EMA and MACD values.

In [ ]:
print("Data:")
print(result[0])

A command is applied to view the lists containing the buy and sell signals. Not all information from the lists is printed. The indices of the lists are also adjusted so that they start with the number 1.

In [ ]:
buy_table = pd.DataFrame({'Row': result[1], 'Signal Date': result[2], 'Buy Date': result[4], 
                          'Buy Price': result[3]}, index=range(1, len(result[1]) + 1))
sell_table = pd.DataFrame({'Row': result[5], 'Signal Date': result[6], 'Sell Date': result[8], 
                           'Sell Price': result[7]}, index=range(1, len(result[5]) + 1))

print("Buy Table:")
print(buy_table)

print("\nSell Table:")
print(sell_table)

A command is excecuted to see the return of each holding period of the stock. In order to accomplish it, a new list is created which stock the return computed by the function. Then, those values are rounded and a new list is made. Besides, all returns are added together to know the performance of the startegy for the specific stock over the decided time period.

In [ ]:
returns_list = result[9]

returns_rounded = [f'{round(return_value, 2)}%' for return_value in returns_list]

performance_table = pd.DataFrame({'Return': returns_rounded}, index=range(1, len(returns_rounded) + 1))

global_performance = sum(returns_list)
                                  
print("Performance Table:")
print(performance_table)
print("\nThe return of this strategy is", f'{round(global_performance, 2)}%')

All the displayed lists can be transferred to a new document in Excel.

In [ ]:
with pd.ExcelWriter('results_MACD.xlsx') as writer:
    result[0].to_excel(writer, sheet_name='Data')
    buy_table.to_excel(writer, sheet_name='Buy Table')
    sell_table.to_excel(writer, sheet_name='Sell Table')
    performance_table.to_excel(writer, sheet_name='Returns')

The first graph shows both MACD and signal line. When both lines cross, it is providing a buy/sell signal.

In [ ]:
result[0][['MACD', 'signal']].plot(
    title= 'MACD and Signal lines',
    grid=True,
    style={'MACD': '-', 'signal': '--'}
)
plt.show()

The second graph shows the price of the price with marker indicating the price at which the stock is bought/sold.

In [ ]:
plt.figure(figsize=(14, 8))
result[0]['Open'].plot(title='Buy and Sell Signals', grid=True)
plt.scatter(result[4], result[3], marker='^', color='green', label='Buy')
plt.scatter(result[8], result[7], marker='v', color='red', label='Sell')
plt.ylabel('Price')
plt.legend()
plt.show()

## 4) References

* Chen, J. (2023, 1 avril). What is EMA ? How to use exponential moving Average with Formula. Investopedia. https://www.investopedia.com/terms/e/ema.asp

* Punwasi, K., & Brijlal, P. (2016). The market reactions to share repurchase announcements on the JSE : an event study. Investment management & financial innovations, 13(1), 191‑205. https://doi.org/10.21511/imfi.13(1-1).2016.06